# **Funzioni utili**
Definiamo alcune funzioni per la suddivisione ed il calcolo di alcune statistiche dei dataset


In [23]:
import argparse
import torch
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, root_mean_squared_error
import shap
from torch.nn import Sigmoid, BCELoss
import torch.optim as optim
import time
from math import inf
from datetime import timedelta
from mlxtend.evaluate import accuracy_score
%run path.ipynb
%run model.ipynb
%run data.ipynb


In [24]:
def train(args):

    # LOAD DATA
    model_name = f'{args.model}_split{args.split}'
 
    train_data = DysgraphiaDL(args.aug, 'train', DEVICE)
    validation_data = DysgraphiaDL(args.aug, 'val', DEVICE)
    
    # LOAD MODEL
    if args.model == 'vit':
        wrapper = ViTWrapper(model_name, DEVICE, 2, False)
    elif args.model == 'resnet':
        wrapper = ResnetWrapper(model_name, DEVICE, 2, False)
    else:
        raise Exception(f'{model} is not a model: selecte either vit or resnet.')
    
    wrapper.binary()
    if args.freeze: wrapper.freeze()
    model = wrapper.get_model()

    print(model)

    # TRAIN SETTINGS
    epochs = 150
    start_epoch = 0
    batch_size = 8
    best_val_loss = inf
    best_val_crit = inf
    exit_counter = 0
    epsilon = 0.001 # counter guard precision improovement
    lr = 0.00001

    act = Sigmoid()
    func = BCELoss()
    loss = lambda x, y: func(torch.reshape(act(x), (-1,)), y.type(torch.float32))

    opt = optim.AdamW(model.parameters(), lr=lr)

    if args.resume_iam:
        start_epoch, best_val_loss, exit_counter, opt_chk, best_val_crit = wrapper.resume(f'{model_name}_checkpoint.pth')
        opt.load_state_dict(opt_chk)

    if args.resume:
        start_epoch, best_val_loss, exit_counter, opt_chk, best_val_crit = wrapper.resume(f'{args.model}_checkpoint.pth')
        opt.load_state_dict(opt_chk)

    print('Start Training')
    for e in range(start_epoch, epochs):

        # TRAIN
        model.train()
        loader = iter(DataLoader(train_data, batch_size=batch_size, shuffle=True))
        train_loss = 0.0
        running_loss = 0.0
        train_crit = 0.0
        running_crit = 0.0

        start = time.time()
        for i in range(0, int(len(train_data) / batch_size) + 1):
            images, classes, pfeat = next(loader)

            opt.zero_grad()
            preds = model(images)

            out = loss(preds, classes)
            out.backward()
            opt.step()

            train_loss += out.item()
            running_loss += out.item()

            classes = np.asarray(classes.cpu())

            preds = torch.reshape(act(preds), (-1,)).cpu().detach().numpy()
            crit = root_mean_squared_error(classes, preds)
            train_crit += crit
            running_crit += crit

            if i % 5 == 4:
                print(f'Epoch {e + 1} - Batch {i + 1}: Running Loss {running_loss / 5} / Running Criteria {running_crit / 5}', end='\r')
                running_loss = 0.0
                running_crit = 0.0
        
        train_loss = train_loss / i
        train_crit = train_crit / i
        print(f'Epoch {e + 1}: Train Loss {train_loss} - Train Criteria {train_crit} - Time {str(timedelta(seconds=time.time() - start))}')

        model.eval()
        with torch.no_grad():
            loader = iter(DataLoader(validation_data, batch_size=len(validation_data), shuffle=False))
            images, classes, pfeat = next(loader)
            preds  = model(images)
            out = loss(preds, classes)
            
            classes = np.asarray(classes.cpu())
           
            preds = torch.reshape(act(preds), (-1,)).cpu()
            crit = root_mean_squared_error(classes, preds)

            val_crit = crit
            print(f"Epoch {e + 1}: Validation Loss {out.item()} - Validation Criteria {val_crit}")
            if val_crit < best_val_crit - epsilon:
                print(f"    !- Validation improovement! {best_val_crit} -> {val_crit}")
                exit_counter = 0
                best_val_crit = val_crit
                is_best = True
            else:
                print(f"    !- No improovement!")
                is_best = False
                exit_counter += 1
            
            state = {
                'epoch': e + 1,
                'state_dict': model.state_dict(),
                'best_val_loss': best_val_loss,
                'exit_counter': exit_counter,
                'optimizer': opt.state_dict(),
                'best_val_crit': best_val_crit
            }
            wrapper.save_state(state, is_best)
        
        if exit_counter == 20:
            print("Exit")
            break

In [25]:
def test(args, explain):

    # LOAD DATA
    model_name = f'{args.model}_split{args.split}'
    test_data = DysgraphiaDL(args.aug, 'test', DEVICE)

    out_classes = 1

    # LOAD MODEL
    if args.model == 'vit':
        wrapper = ViTWrapper(model_name, DEVICE, out_classes)
    elif args.model == 'resnet':
        wrapper = ResnetWrapper(model_name, DEVICE, out_classes, False)
    else:
        raise Exception(f'{model} is not a model: selecte either vit or resnet.')
    
    wrapper.load_state(f'{model_name}_model_best.pth')
    model = wrapper.get_model()

    model.eval()
    with torch.no_grad():
        loader = iter(DataLoader(test_data, batch_size=len(test_data), shuffle=False))
        images, classes, pfeat = next(loader)

        def predict(imgs, pfeat):
            if isinstance(imgs, np.ndarray): imgs = torch.tensor(imgs).to(DEVICE)
            preds  = model(images)
            return preds
        
        out = predict(images, pfeat)

    act = Sigmoid()
    preds = torch.reshape(act(out), (-1,)).cpu()
    classes = np.asarray(classes.cpu())

    print(f"Test Results {model_name}")
    print("---")
    print("Predictions: ", np.asarray(preds.cpu()))
    print("Classes: ", classes)
    print("---")

    preds_binary = (np.array(preds) >= 0.5).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(classes, preds_binary, average='macro')
    accuracy = accuracy_score(classes, preds_binary)

    print("Precision:", round(precision, 3))
    print("Recall:", round(recall, 3))
    print("F1:", round(f1, 3))
    print("Accuracy:", round(accuracy, 3))

    mse = root_mean_squared_error(classes, preds)
    print("Mean Squarred Error:", round(mse, 3))

    if explain:
        topk = 2
        batch_size = 50
        n_evals = 10000
        masker_blur = shap.maskers.Image("blur(128,128)", images[0].shape)
        class_names = {0: 'not-disgraphia', 1: 'disgraphia'}
        def class_to_names(cls):
            names = []
            for c in cls:
                names.append(class_names[c])
            return names
        explainer = shap.Explainer(predict, masker_blur, output_names=['not-disgraphia', 'disgraphia'])
        shap_values = explainer(images[:2], max_evals=n_evals, batch_size=batch_size,
                        outputs=shap.Explanation.argsort.flip[:topk])
        shap_values.data = shap_values.data.cpu().numpy()[0]
        shap_values.values = [val for val in np.moveaxis(shap_values.values,-1, 0)]
        shap.image_plot(shap_values=shap_values.values, pixel_values=shap_values.data, 
                        labels=shap_values.output_names, true_labels=class_to_names(classes))

In [26]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Training')
    parser.add_argument('--resume_iam', '-ri', action="store_true",
                        help="resume training")
    parser.add_argument('--resume', '-r', action="store_true",
                        help="resume training with no_augmented dataset")
    parser.add_argument('--model', '-m', choices=['resnet18', 'vit'], default='resnet',
                        help="set which model to use / train, either 'resnet' or 'vit'")
    parser.add_argument('--aug', '-a', choices=['aug', 'no_aug'], default='aug',
                        help="set which dataset to use, either 'augmented' or 'no_augmented'")
    parser.add_argument('--split', '-s', default=0,
                        help="which train/val/test split to load")
    parser.add_argument('--freeze', '-brr', action="store_true",
                        help="freeze all layers for training but the head")
    parser.add_argument('--explain', '-ex', action="store_true",
                        help="print explainer SHAP results")     
    parser.add_argument('--test', '-t', action="store_true",
                        help="only testing mode")                

    args = parser.parse_args([])
    print(args)

    if not args.test: 
        train(args)
    test(args, explain=args.explain)

Namespace(resume_iam=False, resume=False, model='resnet', aug='no_aug', split=0, freeze=False, explain=False, test=False)
Creating Simple splits.
Creating Simple splits.


C:\Users\renzi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
C:\Users\renzi\AppData\Local\Temp\ipykernel_18964\751997194.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are e

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

C:\Users\renzi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
C:\Users\renzi\AppData\Local\Temp\ipykernel_18964\751997194.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are e

Test Results resnet_split0
---
Predictions:  [0.03590262 0.97330755 0.03804267 0.97476643 0.97411555 0.03460326
 0.97213364 0.0344178  0.9746115  0.97559536 0.03494362 0.974329
 0.03518103 0.03516586 0.9742742  0.97449696 0.9749214  0.97524536
 0.03550141 0.03472495 0.03568916 0.9738791  0.03499717 0.03555688
 0.03572237 0.03439691 0.03512254 0.9739325  0.03536583 0.03451357
 0.9737384  0.9742313  0.03587911 0.9750426  0.9740356  0.9659918
 0.03506077 0.974162   0.03484368 0.03556555 0.97562975 0.04542623
 0.9730833  0.03512604 0.03525349 0.03523761 0.973623   0.03551083
 0.9747146  0.03494941 0.03538159 0.9740967  0.8967888  0.03503603
 0.9736746  0.04709759 0.03571585 0.03481562]
Classes:  [0 1 0 1 1 0 1 0 1 1 0 1 0 0 1 1 1 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 1 1 1 0
 1 0 0 1 0 1 0 0 0 1 0 1 1 0 1 1 0 1 0 0 0]
---
Precision: 0.984
Recall: 0.981
F1: 0.983
Accuracy: 0.983
Mean Squarred Error: 0.131
